### **Initial Set Up**

Install Necessary Packages

In [74]:
import numpy as np
import pandas as pd
import pymysql as msql
import matplotlib.pyplot as plt
import config
import json
import requests
import warnings
import getpass as gp
import mysql.connector as mysql
from datetime import datetime
from mysql.connector import Error
warnings.filterwarnings("ignore", category=DeprecationWarning)

Set up config.py file with subscriber keys necessary from APIs to access the information.

In [6]:

airquality_key = config.a_key
covid_key = config.c_key
weather_key = config.w_key

Password for SQL Connection

In [7]:
passwd=gp.getpass('Enter Password:')

### **Extract and Transform Data**

Insert your City of Choice.

In [134]:
city = 'San Diego'
city

'San Diego'

#### Air Quality API

In [131]:
# URL Website of Air Quality API
url = "https://air-quality-by-api-ninjas.p.rapidapi.com/v1/airquality"

# Parameter Input
querystring = {"city":city}

# Pulling in of Host and Subscriber Key
headers = {
	"X-RapidAPI-Key": airquality_key,
	"X-RapidAPI-Host": "air-quality-by-api-ninjas.p.rapidapi.com"
}

# Pull Data using request library
a_response = requests.request("GET", url, headers=headers, params=querystring)

print(a_response.text)

{"CO": {"concentration": 353.81, "aqi": 4}, "NO2": {"concentration": 45.93, "aqi": 57}, "O3": {"concentration": 35.05, "aqi": 29}, "SO2": {"concentration": 11.21, "aqi": 16}, "PM2.5": {"concentration": 4.5, "aqi": 14}, "PM10": {"concentration": 7.05, "aqi": 6}, "overall_aqi": 57}


In [11]:
# Convert the response to a dictionary
a_data = a_response.json()

# Create a Pandas dataframe from the dictionary
a_df = pd.DataFrame(a_data)
#a_df

In [32]:
# Remove last row as it is not needed.

a_df = a_df.drop(a_df.index[-1])
a_df

,CO,NO2,O3,SO2,PM2.5,PM10,overall_aqi
concentration,283.72,17.99,75.1,3.22,2.23,3.3,100


In [57]:
a_df = a_df.rename(columns={"PM2.5":"FineParticulateMatter", "PM10":"InhalableParticulateMatter", "overall_aqi":"Overall_AQI"})


['CO',
 'NO2',
 'O3',
 'SO2',
 'FineParticulateMatter',
 'InhalableParticulateMatter',
 'Overall_AQI']

In [142]:
# List of Columns in Air Quality and Data Types
print("Variables in Air Quality are: \n", list(a_df.columns))
print("Data Types are: \n",a_df.dtypes)

Variables in Air Quality are: 
 ['CO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'overall_aqi']
Data Types are: 
 CO             float64
NO2            float64
O3             float64
SO2            float64
PM2.5          float64
PM10           float64
overall_aqi      int64
dtype: object


In [22]:
a_dict = a_df.to_dict('records')
a_dict

a_tuple = [tuple(d.values()) for d in a_dict]
a_tuple

[(393.87, 50.72, 36.48, 12.99, 8.06, 10.82, 63),
 (4.0, 63.0, 30.0, 19.0, 26.0, 10.0, 63)]

In [58]:
a_df.to_csv('Data/a_csv.csv')

### Covid-19 API

In [135]:
# make the API call
# link to documentation: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
url = "https://covid-19-statistics.p.rapidapi.com/reports"

# optional parameter selected to querying the API
querystring = {"city_name":city}

# API credentials
headers = {
	"X-RapidAPI-Key": covid_key,
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

# get raw response query from the API
c_response = requests.request("GET", url, headers=headers, params=querystring)
print(c_response.text)

{"data":[{"date":"2023-02-24","confirmed":12084297,"deaths":100816,"recovered":0,"confirmed_diff":3910,"deaths_diff":0,"recovered_diff":0,"last_update":"2023-02-25 04:21:00","active":11983481,"active_diff":3910,"fatality_rate":0.0083,"region":{"iso":"USA","name":"US","province":"California","lat":"36.1162","long":"-119.6816","cities":[{"name":"San Diego","date":"2023-02-24","fips":6073,"lat":"33.03484597","long":"-116.7365326","confirmed":1064093,"deaths":5768,"confirmed_diff":153,"deaths_diff":0,"last_update":"2023-02-25 04:21:00"}]}}]}


In [139]:
# Convert the response to a dictionary
c_data = c_response.json()

# Create a Pandas dataframe from the dictionary
c_df = pd.DataFrame(c_data)

# Unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(c_df['data'])

# merge the new columns back into the original dataframe
df_1 = c_df.merge(df_unpacked, left_index=True, right_index=True)

# drop the original column containing the dictionaries
df_2 = df_1.drop('data', axis=1)

# extract the dictionary from region.cities column for flattening
col_dic = df_2['region.cities'].iloc[0]

# turn the extracted dictionary into its on dataframe 
flatten_col = pd.DataFrame(col_dic)

# merge the two dataframes
result = df_2.merge(flatten_col, left_index=True, right_index=True)

# drop columns 
result.drop(['last_update_x', 'active', 'active_diff', 'region.name', 'region.cities', 'date_y'], axis=1, inplace=True)

# rename columns to be more understandable
c_df = result.rename(columns={"date_x": "date", "confirmed_x": "total_confirmed_cases_state","deaths_x": "total_deaths_state", 
                                         "confirmed_diff_x": "confirmed_today_state", "deaths_diff_x": "deaths_today_state",
                                        "recovered_diff": "recovered_today_state", "fatality_rate":"fatality_rate_state","region.iso":"country",
                                        "region.province": "state", "region.lat":"state_latitude", "region.long":"state_longitude", "name": "city",
                                        "fips":"fips_city_code", "lat": "city_latitude", "long": "city_longitude", "confirmed_y":"total_confirmed_cases_city",
                                        "deaths_y": "total_deaths_city", "confirmed_diff_y":"confirmed_today_city", "deaths_diff_y": "deaths_today_city",
                                        "last_update_y": "last_API_update"})

c_df

,date,total_confirmed_cases_state,total_deaths_state,recovered,confirmed_today_state,deaths_today_state,recovered_today_state,fatality_rate_state,country,state,...,state_longitude,city,fips_city_code,city_latitude,city_longitude,total_confirmed_cases_city,total_deaths_city,confirmed_today_city,deaths_today_city,last_API_update
0,2023-02-24,12084297,100816,0,3910,0,0,0.0083,USA,California,...,-119.6816,San Diego,6073,33.03484597,-116.7365326,1064093,5768,153,0,2023-02-25 04:21:00


In [140]:
# show the values of all the columns
c_dict = c_df.to_dict('records')
#c_dict

c_tuple = [tuple(d.values()) for d in c_dict]
c_tuple

[('2023-02-24',
  12084297,
  100816,
  0,
  3910,
  0,
  0,
  0.0083,
  'USA',
  'California',
  '36.1162',
  '-119.6816',
  'San Diego',
  6073,
  '33.03484597',
  '-116.7365326',
  1064093,
  5768,
  153,
  0,
  '2023-02-25 04:21:00')]

In [143]:
# List of Columns in Covid-19 and Data Types
print("Variables in Covid-19 are: \n", list(c_df.columns))
print("Data Types are: \n",c_df.dtypes)

Variables in Covid-19 are: 
 ['date', 'total_confirmed_cases_state', 'total_deaths_state', 'recovered', 'confirmed_today_state', 'deaths_today_state', 'recovered_today_state', 'fatality_rate_state', 'country', 'state', 'state_latitude', 'state_longitude', 'city', 'fips_city_code', 'city_latitude', 'city_longitude', 'total_confirmed_cases_city', 'total_deaths_city', 'confirmed_today_city', 'deaths_today_city', 'last_API_update']
Data Types are: 
 date                            object
total_confirmed_cases_state      int64
total_deaths_state               int64
recovered                        int64
confirmed_today_state            int64
deaths_today_state               int64
recovered_today_state            int64
fatality_rate_state            float64
country                         object
state                           object
state_latitude                  object
state_longitude                 object
city                            object
fips_city_code                   int64
city

In [141]:
c_csv = c_df.to_csv('Data/c_csv.csv')

### Weather API

In [133]:
# URL Website of Weather API
url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

# Pulling of Host and Subscriber Key
headers = {
	"X-RapidAPI-Key": weather_key,
	"X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
}

# Parameter Input
querystring = {"city": city}

# Pull Data using request library
w_response = requests.request("GET", url, headers=headers, params = querystring)

print(w_response.text)

{"cloud_pct": 75, "temp": 6, "feels_like": 4, "humidity": 89, "min_temp": 5, "max_temp": 8, "wind_speed": 2.57, "wind_degrees": 0, "sunrise": 1677335163, "sunset": 1677375883}


In [121]:
# Convert the response to a dictionary
w_data = w_response.json()

# Create a Pandas dataframe from the dictionary
w_df = pd.DataFrame(w_data, index = [0])

In [122]:
# Convert Unix Timestamp to Readable Datetime
w_df['sunrise'] = datetime.fromtimestamp(w_df['sunrise']).strftime('%Y-%m-%d %H:%M:%S')
w_df['sunset'] = datetime.fromtimestamp(w_df['sunset']).strftime('%Y-%m-%d %H:%M:%S')


In [123]:
# Convert Temperatures from Celsius to Fahrenheit
w_df['temperature'] = w_df.apply(lambda x: (9/5)*x['temp']+32,axis=1)
w_df['temp_feels_like'] = w_df.apply(lambda x: (9/5)*x['feels_like']+32,axis=1)
w_df['min_temperature'] = w_df.apply(lambda x: (9/5)*x['min_temp']+32,axis=1)
w_df['max_temperature'] = w_df.apply(lambda x: (9/5)*x['max_temp']+32,axis=1)

In [124]:
# Preview Dataframe
w_df = w_df.iloc[:,[10, 11, 12, 13, 0, 3, 6, 7, 8, 9]]
w_df

,temperature,temp_feels_like,min_temperature,max_temperature,cloud_pct,humidity,wind_speed,wind_degrees,sunrise,sunset
0,50.0,42.8,44.6,53.6,75,86,8.75,220,2023-02-25 06:21:26,2023-02-25 17:42:19


In [125]:
# List of Columns in Weather and Data Types
print("Variables in Weather are: \n", list(w_df.columns))
print("Data Types are: \n",w_df.dtypes)

Variables in Weather are: 
 ['temperature', 'temp_feels_like', 'min_temperature', 'max_temperature', 'cloud_pct', 'humidity', 'wind_speed', 'wind_degrees', 'sunrise', 'sunset']
Data Types are: 
 temperature        float64
temp_feels_like    float64
min_temperature    float64
max_temperature    float64
cloud_pct            int64
humidity             int64
wind_speed         float64
wind_degrees         int64
sunrise             object
sunset              object
dtype: object


In [126]:
# Convert to Dictionary
w_dict = w_df.to_dict('records')
#w_dict

# Convert to Tuple for feeding into SQL Table
w_tuple = [tuple(d.values()) for d in w_dict]
w_tuple

[(50.0,
  42.8,
  44.6,
  53.6,
  75,
  86,
  8.75,
  220,
  '2023-02-25 06:21:26',
  '2023-02-25 17:42:19')]

In [127]:
w_csv = w_df.to_csv('Data/w_csv.csv')

### **Load Python to SQL Database**

#### Create Database

In [19]:
# Create New Database
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE public_safety")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'public_safety'; database exists


#### Add Air Quality Table to Public Safety Database

In [25]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS air_quality;')
        print('Creating Table Air Quality')
# in the below line please pass the create table statement which you want #to create
        cursor.execute('CREATE TABLE air_quality(CO float, NO2 float, O3 float, SO2 float, FineParticulateMatter float, InhalableParticulateMatter float, Overall_AQI SMALLINT)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        a_sql = 'INSERT INTO air_quality (CO, NO2, O3, SO2, FineParticulateMatter, InhalableParticulateMatter, Overall_AQI) VALUES (%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(a_sql, a_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Air Quality
Table is created....
Record inserted
Table is Complete


In [137]:
query_testing= pd.read_sql(""" 
SELECT * FROM air_quality;
 """, conn)
 
query_testing

,CO,NO2,O3,SO2,FineParticulateMatter,InhalableParticulateMatter,Overall_AQI
0,283.72,17.99,75.1,3.22,2.23,3.3,100


#### Add Weather Table to Public Safety Database

In [128]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS weather;')
        print('Creating Table Weather')
# in the below line please pass the create table statement which you want #to create
        cursor.execute('CREATE TABLE weather(temperature float, temp_feels_like float, min_temperature float, max_temperature float, cloud_pct SMALLINT, humidity SMALLINT, wind_speed float, wind_degrees SMALLINT, sunrise datetime, sunset datetime)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        w_sql = 'INSERT INTO weather (temperature, temp_feels_like, min_temperature, max_temperature, cloud_pct, humidity, wind_speed, wind_degrees, sunrise, sunset) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(w_sql, w_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Weather
Table is created....
Record inserted
Table is Complete


In [129]:
w_query_testing= pd.read_sql(""" 
SELECT * FROM weather;
 """, conn)
 
w_query_testing

,temperature,temp_feels_like,min_temperature,max_temperature,cloud_pct,humidity,wind_speed,wind_degrees,sunrise,sunset
0,50.0,42.8,44.6,53.6,75,86,8.75,220,2023-02-25 06:21:26,2023-02-25 17:42:19


#### Add Covid-19 Table to Public Safety Database

In [148]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS covid;')
        print('Creating Table Covid-19')
# in the below line please pass the create table statement which you want #to create
        cursor.execute('CREATE TABLE covid(date date, total_confirmed_cases_state INT, total_deaths_state INT, recovered INT, confirmed_today_state INT, deaths_today_state INT, recovered_today_state INT, fatality_rate_state float, country text, state text, state_latitude float, state_longitude float, city text, fips_city_code INT, city_latitude float, city_longitude float, total_confirmed_cases_city INT, total_deaths_city INT, confirmed_today_city INT, deaths_today_city INT, last_API_update datetime)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        c_sql = 'INSERT INTO covid (date, total_confirmed_cases_state, total_deaths_state, recovered, confirmed_today_state, deaths_today_state, recovered_today_state, fatality_rate_state, country, state, state_latitude, state_longitude, city, fips_city_code, city_latitude, city_longitude, total_confirmed_cases_city, total_deaths_city, confirmed_today_city, deaths_today_city, last_API_update) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(c_sql, c_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Covid-19
Table is created....
Record inserted
Table is Complete


In [149]:
c_query_testing= pd.read_sql(""" 
SELECT * FROM covid;
 """, conn)
 
c_query_testing

,date,total_confirmed_cases_state,total_deaths_state,recovered,confirmed_today_state,deaths_today_state,recovered_today_state,fatality_rate_state,country,state,...,state_longitude,city,fips_city_code,city_latitude,city_longitude,total_confirmed_cases_city,total_deaths_city,confirmed_today_city,deaths_today_city,last_API_update
0,2023-02-24,12084297,100816,0,3910,0,0,0.0083,USA,California,...,-119.682,San Diego,6073,33.0348,-116.737,1064093,5768,153,0,2023-02-25 04:21:00
